We will use Keras to solve a particle identification problem on simulated data. There are 5 possible outputs, so we are looking at a multi-class classification problem. 

Since we have a multi-class classification problem, each output should be the probability that a particle is x, for some x in {proton, deuteron, triton, he3, he4}. In effect we will turn this into 5 different binary classification problems. The way we do this is with one-hot encoding (I believe this is the right term for what follows). Instead of have a target vector with one column, we want our y samples (targets) to be arranged in a 5 column matrix, where each row will have a single one underneath the column that corresponds to the given output particle type. 

First we upload the packages numpy , tensorflow , and pylab and load the data. 

In [ ]:
import os
import numpy as np
import tensorflow as tf
import pylab as plt
import sklearn as skl
import pandas as pd
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler




In [ ]:
def plot_confusion_matrix_augmented(y_true,
                          y_pred,
                          classes,
                          title=None,
                          cmap=plt.cm.Blues):
    """This function prints and plots the confusion matrix.
    
    Adapted from:
    https://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html
    
    Arguments:
        y_true: Real class labels.
        y_pred: Predicted class labels.
        classes: List of class names.
        title: Title for the plot.
        cmap: Colormap to be used.
    
    Returns:
        None.
    """
    if not title:
        title = 'Confusion matrix'

    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred)

    fig, ax = plt.subplots(figsize=(4, 4), dpi=100)
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)
    # We want to show all ticks...
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           # ... and label them with the respective list entries
           xticklabels=classes, yticklabels=classes,
           title=title,
           ylabel='True label',
           xlabel='Predicted label')

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha='right', rotation_mode='anchor')

    # Loop over data dimensions and create text annotations.
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], 'd'),
                    ha='center', va='center',
                    color='white' if cm[i, j] > thresh else 'black')
    fig.tight_layout()
    plt.show()

In [ ]:
def plot_confusion_matrix_normalized(y_true,
                          y_pred,
                          classes,
                          confusion,
                          title=None,
                          cmap=plt.cm.Blues):
    """This function prints and plots the confusion matrix.
    
    Adapted from:
    https://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html
    
    Arguments:
        y_true: Real class labels.
        y_pred: Predicted class labels.
        classes: List of class names.
        title: Title for the plot.
        cmap: Colormap to be used.
    
    Returns:
        None.
    """
    if not title:
        title = 'Confusion matrix'
    
    
    cm = confusion
    

    fig, ax = plt.subplots(figsize=(4, 4), dpi=100)
    #
    im = ax.imshow( cm.astype(float) , interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)
    # We want to show all ticks...
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           # ... and label them with the respective list entries
           xticklabels=classes, yticklabels=classes,
           title=title,
           ylabel='True label',
           xlabel='Predicted label')

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha='right', rotation_mode='anchor')

    # Loop over data dimensions and create text annotations.
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], '.2f'),
                    ha='center', va='center',
                    color='white' if cm[i, j] > thresh else 'black')
    fig.tight_layout()
    plt.show()

In [ ]:
#we map each particle name to a particle number.
#We will never use the following line of code, but it is helpful to 
#know the convention this codes for mapping each particle to a number.
mapping = {"Proton":0, "Deuteron":1, "Triton":2, "He3": 3, "He4":4}

#We use a pandas dataframe bc it is nicer than numpy array.
#parse -9999 as nan because it doesn't seem like real data:
raw_data = pd.read_csv('Simulation.csv',na_values='-9999')

#taking a look at the current state of the data, with nan values still there:
print(raw_data.head)

#the following comment demonstrates some of the pandas capabilty:
#print(2*raw_data['Px'])

#Here we create new columns in the dataframe:
#For instance, the 'IsProton' column is true
#when the corresponding row is a proton and false otherwise
raw_data['IsProton']= raw_data['Type']=='Proton'
raw_data['IsDeuteron']= raw_data['Type']=='Deuteron'
raw_data['IsTriton']= raw_data['Type']=='Triton'
raw_data['IsHe3']= raw_data['Type']=='He3'
raw_data['IsHe4']= raw_data['Type']=='He4'

#we now see how the above code has added five columns to the dataframe:
print(raw_data.head)

#We have not deleted any rows from the dataframe yet, so lets check its length before cutting:
print('Pre-Clean Length of Data:', len(raw_data))

#we specify in place because we want to make changes to the original copy of 
#raw_data itself, instead of storing this updata in a new object:
#furthermore, dropna drops all rows in the data frame that 
#have at least one nan value 
raw_data.dropna(inplace=True)

print('Check that each column has no nan values. If there') 
print('are no nan values left, the sum will be zero:',np.sum(raw_data.isna()))

#we now expect the dataframe to be a degree shorter due to the cuts we made:
print('Before cutting values x not in range -5000< x < 5000:',len(raw_data))

#the final round of cuts we make will be cutting all data values
#that are less than -5000 and greater than 5000. We do so in two lines of code per feature column:
raw_data = raw_data[raw_data['dEdX'] < 5000]
raw_data = raw_data[raw_data['dEdX'] > -5000]
raw_data = raw_data[raw_data['Px'] < 5000]
raw_data = raw_data[raw_data['Px'] > -5000]
raw_data = raw_data[raw_data['Py'] < 5000]
raw_data = raw_data[raw_data['Py'] > -5000]
raw_data = raw_data[raw_data['Pz'] < 5000]
raw_data = raw_data[raw_data['Pz'] > -5000]
print('After cutting values x not in range -5000< x < 5000:', len(raw_data))

#Let's see the statistics on our data now that we have made the desired edits:
print(raw_data.describe())

#Recall that we have true and false values under each column starting with
#'IsProton'. We want to convert these to integer 0's and 1's.
#We also want to make a new dataframe for this purpose, so that raw_data remains unchanged.
#we use .copy() to ensure no data gets overwritten in raw_data, and .iloc allows
#us to the index the columns by numbers instead of column headers. 
#raw_data_target is a copy of the columns beginning with "IsProton" in raw_data
#raw_data_target = raw_data.iloc[:,7:12].copy()
raw_data_target = raw_data.loc[:,['IsProton', 'IsDeuteron','IsTriton', 'IsHe3', 'IsHe4']].copy()

#Now we turn these booleans into 1's and 0's:
onehot_target_pandas = (1*raw_data_target)

#We check to see if we have a dataframe with 1's and 0's:
print(onehot_target_pandas.head())

#We also need a target dataframe that is not one hot encoded, for the purposes
#of the confusion matrix at the end. Essentially, the neural network will
#create a distribution of predictions, and ...
#target_pandas = raw_data.iloc[:,6].copy().replace({"Proton":0, "Deuteron":1, "Triton":2, "He3": 3, "He4":4})
target_pandas = raw_data.loc[:,['Type']].copy().replace({"Proton":0, "Deuteron":1, "Triton":2, "He3": 3, "He4":4})

print(target_pandas.head())
#do a histogram. 
#log scale the column data. 
#add a number to each column. 

#finally, for use later on, lets deignate a dataframe that has just the 
#feature columns, dEdX, Px, Py, and Pz:
#design_pandas = raw_data.iloc[:,0:4].copy()
design_pandas = raw_data.loc[:,['dEdX','Px','Py','Pz']].copy()
print(design_pandas.head())

In [ ]:
#we want to see the dedX data in a histogram:
plt.subplot(221)
plt.hist(raw_data['Px'],log=True, bins=40)
plt.subplot(222)
plt.hist(raw_data['Py'],log=True, bins=40)
plt.subplot(223)
plt.hist(raw_data['Pz'],log=True, bins=40)
plt.subplot(224)
plt.hist(raw_data['dEdX'],log=True, bins=40)

In [ ]:
#we want to see the dedX data in a histogram:
plt.subplot(221)
plt.hist(np.log10(raw_data['Px']+5001),log=True, bins=40)
plt.subplot(222)
plt.hist(np.log10(raw_data['Py']+5001),log=True, bins=40)
plt.subplot(223)
plt.hist(np.log10(raw_data['Pz']+2),log=True, bins=40)
plt.subplot(224)
plt.hist(np.log10(raw_data['dEdX']+2),log=True, bins=40)

In [ ]:
plt.hist(raw_data['Type'])

Now we create out design matrix, which we will call 'design'. Here, we want each row to contain an example, and each example to be described by it's dedx, px,py, and pz

In [ ]:
#Let's convert the following pandas dataframes to numpy arrays:
# 1 - target_pandas
# 2 - onehot_target_pandas
# 3 - design_pandas
nponehot_target = onehot_target_pandas.to_numpy()

design = design_pandas.to_numpy()

#normalize the data using the minmaxscaler from sklearn. This is a class. 
scaler = MinMaxScaler()

design = scaler.fit_transform(design)
#design = skl.preprocessing.normalize(design)

target = target_pandas.to_numpy()
        

In [ ]:
#we want to see the dedX data in a histogram:
plt.subplot(221)
plt.hist(design[:,0],log=True, bins=40)
plt.subplot(222)
plt.hist(design[:,1],log=True, bins=40)
plt.subplot(223)
plt.hist(design[:,2],log=True, bins=40)
plt.subplot(224)
plt.hist(design[:,3],log=True, bins=40)

In [ ]:
print(len(nponehot_target))
print(len(nponehot_target)/5, " examples for testing")
print(len(nponehot_target)- len(nponehot_target)/5)

In [ ]:
#training data:
TRAIN_SPLIT_INDEX = 532494

y_train = nponehot_target[:TRAIN_SPLIT_INDEX]
x_train = design[:TRAIN_SPLIT_INDEX]
y_train_non_one_hot = target[:TRAIN_SPLIT_INDEX]

#test data:
x_test  = design[TRAIN_SPLIT_INDEX:]
y_test  = target[TRAIN_SPLIT_INDEX:]

print('Training Data:')
print(x_train)
print(y_train)

print('Testing Data')
print(x_test)
print(y_test)


Here is the code for plotting validation loss and training loss:

In [ ]:
def plot_learning_curve(history):
    plt.plot(history["loss"], label="training loss")
    plt.plot(history["val_loss"], label="validation loss")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.yscale('log')
    plt.legend()
    plt.show()



In [ ]:
# This is the directory where model weights will be saved. Feel free to change it.
CHECKPOINT_DIR = './model-checkpoints-PID/'

if not os.path.exists(CHECKPOINT_DIR):
    os.makedirs(CHECKPOINT_DIR)

# The checkpoints will be saved with the corresponding epoch number in their filename
ckpt_path = os.path.join(CHECKPOINT_DIR, 'weights.epoch.{epoch:02d}')

# Setup checkpoint callback. We only save the weights, not the entire model
ckpt_callback = tf.keras.callbacks.ModelCheckpoint(ckpt_path, save_weights_only=True)

In [ ]:
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau("loss", 
                                                 factor=0.5, patience=5, min_lr = 0.00000001, mode="min") 

In [ ]:
my_callbacks = [
    tf.keras.callbacks.EarlyStopping(monitor="loss", patience=10),
    ckpt_callback,
    reduce_lr
]

Let's use relu because we have a multi class logistic regression problem at hand. Let's start out with 3 hidden layers with 30 neurons, an output layer with 5 output options (because there were no Pions), and let's use categoricalcrossentropy because that seems fair enough (though we could switch to the log error from logistic regression videos later on). Also, let our validation split be 0.2, meaning we train on 800 examples and validate on 200. Let's start with a batch size of 64 and train for 300 epochs. 

In [ ]:

model = tf.keras.Sequential() #Define the model object
model.add(tf.keras.layers.Dense(30, input_shape=(4,), activation="relu")) #Add the hidden layer
model.add(tf.keras.layers.Dense(30, activation="relu"))
model.add(tf.keras.layers.Dense(30, activation="relu"))

#output layer:
model.add(tf.keras.layers.Dense(5, activation="softmax"))


model.compile(tf.keras.optimizers.Adam(lr=0.005),loss=tf.keras.losses.CategoricalCrossentropy()) #Adam optimizer and mean squared error loss

In [ ]:

results = model.fit(x_train, y_train, epochs=300, batch_size=64, validation_split=0.2, callbacks=my_callbacks)

plot_learning_curve(results.history)

In [ ]:
EARLY_STOPPING_EPOCH = 193

assert EARLY_STOPPING_EPOCH > 0, 'You need to set an early stopping point!'

# Path the the checkpoint we want to load
es_ckpt_path = os.path.join(CHECKPOINT_DIR, 'weights.epoch.{:02d}'.format(EARLY_STOPPING_EPOCH))

# Load the weights from the desired checkpoint into the model
model.load_weights(es_ckpt_path);

In [ ]:
predictions = model.predict(x_test)

print(predictions[:5])
predictions = np.argmax(predictions, axis=1)
print(predictions[:5])


In [ ]:
def print_confusion_matrix_1(y_test,predictions,normalize='true'):
    confusion = confusion_matrix(y_test,predictions,normalize=normalize)
    print('Recall')
    print(confusion)
print_confusion_matrix_1(y_test,predictions)

In [ ]:
def print_confusion_matrix_2(y_test,predictions,normalize='pred'):
    confusion = confusion_matrix(y_test,predictions,normalize=normalize)
    print('Precision')
    print(confusion)
print_confusion_matrix_2(y_test,predictions)

In [ ]:
plot_confusion_matrix_augmented(y_test, predictions, ["Protons", "Deuterons", "Triton","He3", "He4"])


In [ ]:
normalize='pred'

confusion = confusion_matrix(y_test, predictions, normalize=normalize)

plot_confusion_matrix_normalized(y_test, predictions, ["Protons", "Deuterons", "Triton","He3", "He4"], confusion)